In [1]:
import polars as pl
import pandas as pd
import numpy as np
from pathlib import Path
import csv
import tqdm
import os


from IPython.display import display

In [2]:
# Usage

def preprocess(file_path, output_file):
    # Determine total file size for progress tracking
    file_size = os.path.getsize(file_path)
    # max_lines = 100000
    line_index = 0  
    bad_rows = 0
    with open(file_path, 'rb') as infile, open(output_file, 'w', newline='', encoding='utf-8') as outfile:
        writer = csv.writer(outfile)
        progress = tqdm.tqdm(total=file_size, desc="Processing File", unit="B", unit_scale=True)

        # Initialize control variables
        current_row = []
        processed_bytes = 0

        # Read and process the file line by line in binary mode
        while True:
            line = infile.readline()
            if not line:
                break

            processed_bytes += len(line)
            progress.update(len(line))
            
            # Decode line, handle escaped tabs and internal newlines
            decoded_line = line.decode('utf-8')
            decoded_line = decoded_line.replace('\\\\t', '<ESC_TAB>').replace('\r\n', '<ESC_NEWLINE>').replace('\r', '<ESC_CR>')

            # Split on unescaped tabs, process each part
            parts = decoded_line.split('\t')
            for part in parts:
                normalized_part = part.replace('<ESC_TAB>', '\t').replace('<ESC_NEWLINE>', '\r\n').replace('<ESC_CR>', '\r').strip()
                current_row.append(normalized_part)

            # If the line ends with a newline (split condition), it means end of a row
            if decoded_line.endswith('\n'):
                # Write the completed row to the file
                if len(current_row) != 51:
                    bad_rows += 1
                writer.writerow(current_row)
                current_row = []
            else:
                # print(f"Line {line_index} does not end with a newline, continuing to next line")
                pass

            line_index += 1
            # if line_index > max_lines:
            #     break

        # Write any remaining data in current_row to the file (in case the last line does not end with a newline)
        if current_row:
            writer.writerow(current_row)

        progress.close()
    print(f"Bad rows: {bad_rows}/{line_index} ({bad_rows/line_index*100:.2f}%)")

apollo_file_path = Path('data/apollo/Apollo 200 Million 3_3-008.csv')
apollo_output_file = Path('data/apollo/Apollo 200 Million 3_3-008_corrected_2.csv')

if not apollo_output_file.exists():
    preprocess(apollo_file_path, apollo_output_file)
    
    
apollo2_file_path = Path('data/apollo/Apollo 200 Million 2_3-002.csv')
apollo2_output_file = Path('data/apollo/Apollo 200 Million 2_3-002_corrected.csv')

if not apollo2_output_file.exists():
    preprocess(apollo2_file_path, apollo2_output_file)


In [3]:
apollo_companies_df = pl.read_csv('data/apollo/Apollo 200 Million 3_3-008_corrected_2.csv', truncate_ragged_lines=True, infer_schema_length=0)

In [4]:
apollo_people_df = pl.read_csv('data/apollo/Apollo 200 Million 2_3-002.csv', truncate_ragged_lines=True, infer_schema_length=0)

In [19]:
combined_df = pl.concat([pl.read_csv(file, ignore_errors=True) for file in corrected_csv_files])

In [20]:
print(f"Bad rows {combined_df['LEGALENTITYCONGRESSIONALDISTRICT'].is_null().sum()}")

# Drop rows with missing 'LEGALENTITYCONGRESSIONALDISTRICT' values
combined_df = combined_df.filter(combined_df['LEGALENTITYCONGRESSIONALDISTRICT'].is_not_null())

print(f"Number of rows after dropping missing 'LEGALENTITYCONGRESSIONALDISTRICT' values: {len(combined_df)}")

Bad rows 850
Number of rows after dropping missing 'LEGALENTITYCONGRESSIONALDISTRICT' values: 3765391


In [3]:
import os
import tqdm

def check_line_endings(file_path):
    line_endings = {
        'LF': 0,
        'CRLF': 0,
        'CR': 0,
        'None': 0  # To handle files that might not end with a newline character
    }

    file_size = os.path.getsize(file_path)
    processed_bytes = 0

    with open(file_path, 'rb') as file:
        # Set up the tqdm progress bar
        with tqdm.tqdm(total=file_size, desc="Processing file", unit="B", unit_scale=True) as pbar:
            while True:
                line = file.readline()
                if not line:
                    break  # End of file

                processed_bytes += len(line)
                pbar.update(len(line))

                # Check line endings
                if line.endswith(b'\r\n'):
                    line_endings['CRLF'] += 1
                elif line.endswith(b'\n'):
                    line_endings['LF'] += 1
                elif line.endswith(b'\r'):
                    line_endings['CR'] += 1

    for ending, count in line_endings.items():
        print(f"Lines ending with {ending}: {count}")

# Usage
file_path = 'data/apollo/Apollo 200 Million 3_3-008.csv'
check_line_endings(file_path)


Processing file: 100%|██████████| 10.2G/10.2G [00:10<00:00, 967MB/s] 

Lines ending with LF: 5593564
Lines ending with CRLF: 475969
Lines ending with CR: 0
Lines ending with None: 0


In [9]:
import os
import tqdm

def check_line_endings(file_path, buffer_size=100000):
    line_endings = {
        'LF': 0,
        'CRLF': 0,
        'CR': 0,
        'None': 0
    }

    
    file_size = os.path.getsize(file_path)
    max_file_size = 100000000

    file_size = min(file_size, max_file_size)
    total_chunks = (file_size // buffer_size) + (1 if file_size % buffer_size != 0 else 0)


    last_char = b''
    with open(file_path, 'rb') as file:
        # Set up the tqdm progress bar
        with tqdm.tqdm(total=total_chunks, desc="Processing file") as pbar:
            buffer = file.read(buffer_size)
            while buffer:
                for char in buffer:
                    if char == ord('\n'):
                        if last_char == ord('\r'):
                            line_endings['CRLF'] += 1
                        else:
                            line_endings['LF'] += 1
                    elif last_char == ord('\r'):  # This carriage return was not followed by a newline
                        line_endings['CR'] += 1
                        print(f"CR found at {file.tell()} on line {line_endings['LF'] - line_endings['CRLF']}")
                    last_char = char

                buffer = file.read(buffer_size)
                pbar.update(1)  # Update progress bar after processing each buffer
                if file.tell() > max_file_size:
                    break

    # Check if the very last character was a CR that wasn't followed by a newline
    if last_char == ord('\r'):
        line_endings['CR'] += 1

    for ending, count in line_endings.items():
        print(f"Lines ending with {ending}: {count}")

# Usage
file_path = 'data/apollo/Apollo 200 Million 3_3-008.csv'
check_line_endings(file_path)


Processing file:  13%|█▎        | 128/1000 [00:01<00:07, 118.06it/s]

CR found at 11300000 on line 4454
CR found at 12200000 on line 4814
CR found at 12200000 on line 4814


Processing file:  24%|██▎       | 237/1000 [00:02<00:06, 115.34it/s]

CR found at 22400000 on line 8825
CR found at 22400000 on line 8825
CR found at 22400000 on line 8825
CR found at 22400000 on line 8825
CR found at 22400000 on line 8825
CR found at 22400000 on line 8825


Processing file:  57%|█████▊    | 575/1000 [00:05<00:03, 115.14it/s]

...

CR found at 67100000 on line 25543


Processing file:  78%|███████▊  | 779/1000 [00:06<00:01, 113.14it/s]

CR found at 76300000 on line 29026
CR found at 76300000 on line 29026
CR found at 76300000 on line 29026


Processing file: 100%|██████████| 1000/1000 [00:08<00:00, 113.65it/s]

CR found at 97200000 on line 37157
CR found at 97200000 on line 37157
CR found at 97200000 on line 37157
CR found at 97200000 on line 37157
Lines ending with LF: 41346
Lines ending with CRLF: 3042
Lines ending with CR: 18
Lines ending with None: 0


In [4]:
def preprocess(file_path, output_file):
    # Determine total file size for progress tracking
    file_size = os.path.getsize(file_path)
    # max_lines = 100000
    line_index = 0  
    bad_rows = 0
    with open(file_path, 'rb') as infile, open(output_file, 'w', newline='', encoding='utf-8') as outfile:
        writer = csv.writer(outfile)
        progress = tqdm.tqdm(total=file_size, desc="Processing File", unit="B", unit_scale=True)

        # Initialize control variables
        current_row = []
        processed_bytes = 0

        # Read and process the file line by line in binary mode
        while True:
            line = infile.readline()
            if not line:
                break

            processed_bytes += len(line)
            progress.update(len(line))
            
            # Decode line, handle escaped tabs and internal newlines
            decoded_line = line.decode('utf-8')
            decoded_line = decoded_line.replace('\\\\t', '<ESC_TAB>').replace('\r\n', '<ESC_NEWLINE>').replace('\r', '<ESC_CR>')

            # Split on unescaped tabs, process each part
            parts = decoded_line.split('\t')
            for part in parts:
                normalized_part = part.replace('<ESC_TAB>', '\t').replace('<ESC_NEWLINE>', '\r\n').replace('<ESC_CR>', '\r').strip()
                current_row.append(normalized_part)

            # If the line ends with a newline (split condition), it means end of a row
            if decoded_line.endswith('\n'):
                # Write the completed row to the file
                if len(current_row) != 51:
                    bad_rows += 1
                writer.writerow(current_row)
                current_row = []
            else:
                # print(f"Line {line_index} does not end with a newline, continuing to next line")
                pass

            line_index += 1
            # if line_index > max_lines:
            #     break

        # Write any remaining data in current_row to the file (in case the last line does not end with a newline)
        if current_row:
            writer.writerow(current_row)

        progress.close()
    print(f"Bad rows: {bad_rows}/{line_index} ({bad_rows/line_index*100:.2f}%)")

# Usage
file_path = 'data/apollo/Apollo 200 Million 3_3-008.csv'
output_file = 'data/apollo/Apollo 200 Million 3_3-008_corrected_2.csv'
preprocess(file_path, output_file)


Processing File: 100%|██████████| 10.2G/10.2G [04:09<00:00, 40.9MB/s]


Bad rows: 331105/6069533 (5.46%)


In [24]:
apollo_df = pl.read_csv('data/apollo/Apollo 200 Million 3_3-008_corrected_2.csv', truncate_ragged_lines=True, infer_schema_length=0)

In [25]:
apollo_df['_score'].value_counts(sort=True)

_score,count
str,u32
"""1""",5262931
null,201116
"""person""",26833
"""people_v7""",18247
"""contact""",9704
…,…
"""593cab21a6da9820b5781b21""",1
"""57db9b6ca6da986863071df7""",1
"""{'type': 'envelope', 'coordina…",1


In [8]:
pd.read_csv('data/apollo/Apollo 200 Million 3_3-008_corrected_2.csv')

,organization_id,organization_name,organization_revenue_in_thousands_int,organization_retail_location_count,organization_public_symbol,organization_linkedin_company_size_tag_ids,organization_founded_year,organization_alexa_ranking,organization_num_current_employees,organization_relevant_keywords,...,organization_hq_location_postal_code,organization_hq_location_geojson,organization_total_funding_long,organization_latest_funding_stage_cd,organization_latest_funding_round_amount_long,organization_latest_funding_round_date,_index,_type,_id,_score
0,54a11ef769702d8ccc8ab201,United Nations,NaN,NaN,NaN,['5567e0c37369641233df0600'],1945,1627,21789,"['development', 'peacekeeping', 'peace & secur...",...,10017,"{'type': 'envelope', 'coordinates': [[-73.9682...",NaN,NaN,NaN,NaN,people_v7,person,54a98ccd74686930ad4b4714,1
1,54a1366a69702d46f00b1f01,Bina Nusantara University,35000,NaN,NaN,['5567cd517369643981040000'],1974,31496,1175,"['accounting', 'architecture', 'art design', '...",...,NaN,"{'type': 'envelope', 'coordinates': [[106.7822...",NaN,NaN,NaN,NaN,people_v7,person,54a586547468692fa266469a,1
2,54a12ae469702d9313fb6402,Despacho Presidencial,NaN,NaN,NaN,['5567cd4a73696453e12f0000'],NaN,NaN,89,['government administration'],...,NaN,"{'type': 'envelope', 'coordinates': [[-74.9376...",NaN,NaN,NaN,NaN,people_v7,person,54ebba33746869444cfd4117,1
3,54a12af269702d9b8b8d5e02,Independiente,NaN,NaN,NaN,['5567e10673696418751f0100'],NaN,NaN,89754,['photography'],...,NaN,NaN,NaN,NaN,NaN,NaN,people_v7,person,54c1fb3074686916396e6d66,1
4,54a127c769702d8cfc25f600,Banco Bci,1800000,NaN,NaN,['5567e0c37369641233df0600'],1937,12353,4360,"['financial service', 'investors', 'banking', ...",...,NaN,"{'type': 'envelope', 'coordinates': [[-70.5561...",NaN,NaN,NaN,NaN,people_v7,person,54a226aa74686930c2762311,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5515454,NaN,people_v7,person,54a9ef4d7468693399bb4b1b,1,55e85115f3e5bb7f1e000539,Bittersweet Acres,NaN,NaN,NaN,...,LaGrange,"LaGrange, Indiana",Indiana,"Indiana, US",United States,46761,"{'type': 'envelope', 'coordinates': [[-85.4247...",NaN,NaN,NaN
5515455,NaN,people_v7,person,54c229d17468697af7abcd80,1,56d4e47cf3e5bb07050018f6,"DingSen Industry Co., Ltd",NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5515456,NaN,people_v7,person,57df4d10a6da980ad8f08660,1,54a134f769702d3e830b6900,Colors of Life,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5515457,NaN,people_v7,person,54a1275569702db878b42e00,1,5592101d736964186fd84800,ZEMS ENTERTAINMENT LIMITED,NaN,NaN,NaN,...,London,"London, United Kingdom",England,"England, United Kingdom",United Kingdom,E7 9AL,"{'type': 'envelope', 'coordinates': [[0.013332...",NaN,NaN,NaN


In [21]:
# apollo_dir = data_dir / 'apollo'

# apollo_csv_files = []

# for csv_file in apollo_dir.rglob('*.csv'):
#     apollo_csv_files.append(csv_file)
    

# print(f"Number of Apollo CSV files: {len(apollo_csv_files)}")

# # Function to read only the first row (header) of a CSV to check consistency
# def read_header(file):
#     try:
#         df = pl.read_csv(file, has_header=True, new_columns=None, n_rows=0, infer_schema_length=0)
#         return df.schema
#     except Exception as e:
#         print(f"Error reading {file}: {e}")
#         return None
    
# headers = [read_header(file) for file in apollo_csv_files]
# first_header = headers[0]

# for i, header in enumerate(headers):
#     if header != first_header:
#         print(f"Header {i} do not match across files.")
#         break

# if all(header == first_header for header in headers):
#     print("All headers match.")
# else:
#     raise ValueError("Headers do not match across files.")

# # Function to read a CSV file with all data as string
# def load_csv_as_string(file):
#     try:
#         return pl.read_csv(file, has_header=True, infer_schema_length=0)
#     except Exception as e:
#         print(f"Failed to load {file}: {e}")
#         return None

# # Load all data treating everything as strings and concatenate
# appolo_df = pl.concat([pl.read_csv(file, has_header=True, infer_schema_length=0) for file in apollo_csv_files])#, how='diagonal')

# print(f"Total rows in concatenated dataframe: {len(appolo_df)}")

In [31]:
interesting_cols = ['AWARDEEORRECIPIENTLEGALENTITYNAME', 'LEGALENTITYADDRLINE1', 'LEGALENTITYCITYNAME', 'LEGALENTITYSTATECD', 'LEGALENTITYZIP5']

In [9]:
combined_df

ACTIONTYPE,ACTIONDATE,ASSISTANCETYPE,RECORDTYPE,FAIN,AWARDMODIFICATIONAMENDMENTNUM,URI,CORRECTIONLATEDELETEIND,FISCALYEARANDQTRCORRECTION,SAI_NUM,AWARDEEORRECIPIENTLEGALENTITYNAME,AWARDEEORRECIPIENTUNIQUEIDENTIFIER,LEGALENTITYADDRLINE1,LEGALENTITYADDRLINE2,LEGALENTITYADDRLINE3,LEGALENTITYCITYNAME,LEGALENTITYSTATECD,LEGALENTITYZIP5,LEGALENTITYZIPLAST4,LEGALENTITYCOUNTRYCD,LEGALENTITYFOREIGNCITYNAME,LEGALENTITYFOREIGNPROVINCENAME,LEGALENTITYFOREIGNPOSTALCD,LEGALENTITYCONGRESSIONALDISTRICT,BUSINESSTYPES,FUNDINGAGENCYCD,FUNDINGSUBTIERAGENCYCD,FUNDINGOFFICECD,AWARDINGAGENCYCD,AWARDINGSUBTIERAGENCYCD,AWARDINGOFFICECD,CFDA_NUM,PRIMPLACEOFPERFORMANCECD,PRIMPLACEOFPERFORMANCECOUNTRYCD,PRIMPLACEOFPERFORMANCEZIP+4,PRIMPLACEOFPERFORMANCEFOREIGNLOCATIONDESC,PRIMPLACEOFPERFORMANCECONGRESSIONALDISTRICT,AWARDDESC,PERIODOFPERFORMANCESTARTDATE,PERIODOFPERFORMANCECURRENTENDDATE,FEDERALACTIONOBLIGATION,NONFEDERALFUNDINGAMOUNT,FACEVALUEOFDIRECTLOANORLOANGUARANTEE,ORIGINALLOANSUBSIDYCOST,BUSINESSFUNDSINDICATOR
str,i64,i64,i64,i64,i64,str,str,str,str,str,str,str,str,str,str,str,i64,i64,str,str,str,str,i64,str,i64,i64,i64,i64,i64,i64,f64,str,str,str,str,i64,str,str,str,str,str,f64,f64,str
"""A""",20200803,7,2,2443918208,1,null,null,null,"""SAI Exempt""","""AKIKO'S SUSHI BAR INC.""",null,"""726 Noriega St""",null,null,"""SAN FRANCISCO""","""CA""",94122,4542,"""USA""",null,null,null,12,"""R""",73,7300,732990,73,7300,732990,59.008,"""CA**075""","""USA""",null,null,12,"""TO PROVIDE LOANS TO RESTORE AS…",null,null,null,null,144000.0,19612.8,"""NON"""
"""A""",20200803,7,2,2446098206,1,null,null,null,"""SAI Exempt""","""DITZLER GENERAL CONTRACTING IN…",null,"""212 WILE AVE""",null,null,"""SOUDERTON""","""PA""",18964,1624,"""USA""",null,null,null,1,"""R""",73,7300,732990,73,7300,732990,59.008,"""PA**091""","""USA""",null,null,1,"""TO PROVIDE LOANS TO RESTORE AS…",null,null,null,null,25300.0,3445.86,"""NON"""
"""A""",20200803,7,2,2448018208,1,null,null,null,"""SAI Exempt""","""HERMAN CLEANERS, LLC""",null,"""5590 KEYSTONE PINE WAY""",null,null,"""DUBLIN""","""OH""",43016,9472,"""USA""",null,null,null,12,"""R""",73,7300,732990,73,7300,732990,59.008,"""OH**049""","""USA""",null,null,12,"""TO PROVIDE LOANS TO RESTORE AS…",null,null,null,null,100000.0,13620.0,"""NON"""
"""A""",20200803,7,2,2449338201,1,null,null,null,"""SAI Exempt""","""RSM REALTY, LLC""",null,"""706 OXFORD RD""",null,null,"""YPSILANTI""","""MI""",48197,2146,"""USA""",null,null,null,12,"""R""",73,7300,732990,73,7300,732990,59.008,"""MI**161""","""USA""",null,null,12,"""TO PROVIDE LOANS TO RESTORE AS…",null,null,null,null,28600.0,3895.32,"""NON"""
"""A""",20200803,7,2,2455458209,1,null,null,null,"""SAI Exempt""","""PINK PALETTE ARTISTS LLC""",null,"""7738 HERON LAKES DR""",null,null,"""HOUSTON""","""TX""",77064,1739,"""USA""",null,null,null,2,"""R""",73,7300,732990,73,7300,732990,59.008,"""TX**201""","""USA""",null,null,2,"""TO PROVIDE LOANS TO RESTORE AS…",null,null,null,null,98400.0,13402.08,"""NON"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""A""",20200828,7,2,7278918206,1,null,null,null,"""SAI Exempt""","""Saiko Excavation LLC""",null,"""322 S 400 W""",null,null,"""Logan""","""UT""",84321,5130,"""USA""",null,null,null,1,"""R""",73,7300,732990,73,7300,732990,59.008,"""UT**005""","""USA""",null,null,1,"""TO PROVIDE LOANS TO RESTORE AS…",null,null,null,null,25000.0,3405.0,"""NON"""
"""A""",20200828,7,2,7174428207,1,null,null,null,"""SAI Exempt""","""Baryonnoh Vision""",null,"""5982 BROOKSTONE DR NW""",null,null,"""CONCORD""","""NC""",28027,2533,"""USA""",null,null,null,8,"""MR""",73,7300,732990,73,7300,732990,59.008,"""NC**025""","""USA""",null,null,8,"""TO PROVIDE LOANS TO RESTORE AS…",null,null,null,null,15000.0,2043.0,"""NON"""
"""A""",20200827,7,2,6622258204,1,null,null,null,"""SAI Exempt""","""EW Real Estate LLC""",null,"""3630 concord dr""",null,null,"""beachwood""","""OH""",44122,6015,"""USA""",null,null,null,11,"""R""",73,7300,732990,73,73

In [23]:
import fuzzywuzzy

In [22]:
combined_df['AWARDEEORRECIPIENTLEGALENTITYNAME']

AWARDEEORRECIPIENTLEGALENTITYNAME
str
"""AKIKO'S SUSHI BAR INC."""
"""DITZLER GENERAL CONTRACTING IN…"
"""HERMAN CLEANERS, LLC"""
"""RSM REALTY, LLC"""
"""PINK PALETTE ARTISTS LLC"""
…
"""Saiko Excavation LLC"""
"""Baryonnoh Vision"""
"""EW Real Estate LLC"""


In [26]:
apollo_df['organization_name']

organization_name
str
"""United Nations"""
"""Bina Nusantara University"""
"""Despacho Presidencial"""
"""Independiente"""
"""Banco Bci"""
…
"""Bittersweet Acres"""
"""DingSen Industry Co., Ltd"""
"""Colors of Life"""


In [27]:
# sampled_name = combined_df['AWARDEEORRECIPIENTLEGALENTITYNAME'].sample(n=100, with_replacement=False)#.apply(lambda x: x.lower())
sampled_name = combined_df['AWARDEEORRECIPIENTLEGALENTITYNAME'].head()
# lower case in polar
sampled_names_lower_case = sampled_name.str.to_lowercase()

company_names_lower_case = apollo_df['organization_name'].str.to_lowercase()

In [28]:
sampled_names_lower_case

AWARDEEORRECIPIENTLEGALENTITYNAME
str
"""akiko's sushi bar inc."""
"""ditzler general contracting in…"
"""herman cleaners, llc"""
"""rsm realty, llc"""
"""pink palette artists llc"""
"""mainsource llc"""
"""beth preddy public relations i…"
"""avenir thinking, inc."""
"""herman green"""


In [29]:
combined_df.head()

ACTIONTYPE,ACTIONDATE,ASSISTANCETYPE,RECORDTYPE,FAIN,AWARDMODIFICATIONAMENDMENTNUM,URI,CORRECTIONLATEDELETEIND,FISCALYEARANDQTRCORRECTION,SAI_NUM,AWARDEEORRECIPIENTLEGALENTITYNAME,AWARDEEORRECIPIENTUNIQUEIDENTIFIER,LEGALENTITYADDRLINE1,LEGALENTITYADDRLINE2,LEGALENTITYADDRLINE3,LEGALENTITYCITYNAME,LEGALENTITYSTATECD,LEGALENTITYZIP5,LEGALENTITYZIPLAST4,LEGALENTITYCOUNTRYCD,LEGALENTITYFOREIGNCITYNAME,LEGALENTITYFOREIGNPROVINCENAME,LEGALENTITYFOREIGNPOSTALCD,LEGALENTITYCONGRESSIONALDISTRICT,BUSINESSTYPES,FUNDINGAGENCYCD,FUNDINGSUBTIERAGENCYCD,FUNDINGOFFICECD,AWARDINGAGENCYCD,AWARDINGSUBTIERAGENCYCD,AWARDINGOFFICECD,CFDA_NUM,PRIMPLACEOFPERFORMANCECD,PRIMPLACEOFPERFORMANCECOUNTRYCD,PRIMPLACEOFPERFORMANCEZIP+4,PRIMPLACEOFPERFORMANCEFOREIGNLOCATIONDESC,PRIMPLACEOFPERFORMANCECONGRESSIONALDISTRICT,AWARDDESC,PERIODOFPERFORMANCESTARTDATE,PERIODOFPERFORMANCECURRENTENDDATE,FEDERALACTIONOBLIGATION,NONFEDERALFUNDINGAMOUNT,FACEVALUEOFDIRECTLOANORLOANGUARANTEE,ORIGINALLOANSUBSIDYCOST,BUSINESSFUNDSINDICATOR
str,i64,i64,i64,i64,i64,str,str,str,str,str,str,str,str,str,str,str,i64,i64,str,str,str,str,i64,str,i64,i64,i64,i64,i64,i64,f64,str,str,str,str,i64,str,str,str,str,str,f64,f64,str
"""A""",20200803,7,2,2443918208,1,null,null,null,"""SAI Exempt""","""AKIKO'S SUSHI BAR INC.""",null,"""726 Noriega St""",null,null,"""SAN FRANCISCO""","""CA""",94122,4542,"""USA""",null,null,null,12,"""R""",73,7300,732990,73,7300,732990,59.008,"""CA**075""","""USA""",null,null,12,"""TO PROVIDE LOANS TO RESTORE AS…",null,null,null,null,144000.0,19612.8,"""NON"""
"""A""",20200803,7,2,2446098206,1,null,null,null,"""SAI Exempt""","""DITZLER GENERAL CONTRACTING IN…",null,"""212 WILE AVE""",null,null,"""SOUDERTON""","""PA""",18964,1624,"""USA""",null,null,null,1,"""R""",73,7300,732990,73,7300,732990,59.008,"""PA**091""","""USA""",null,null,1,"""TO PROVIDE LOANS TO RESTORE AS…",null,null,null,null,25300.0,3445.86,"""NON"""
"""A""",20200803,7,2,2448018208,1,null,null,null,"""SAI Exempt""","""HERMAN CLEANERS, LLC""",null,"""5590 KEYSTONE PINE WAY""",null,null,"""DUBLIN""","""OH""",43016,9472,"""USA""",null,null,null,12,"""R""",73,7300,732990,73,7300,732990,59.008,"""OH**049""","""USA""",null,null,12,"""TO PROVIDE LOANS TO RESTORE AS…",null,null,null,null,100000.0,13620.0,"""NON"""
"""A""",20200803,7,2,2449338201,1,null,null,null,"""SAI Exempt""","""RSM REALTY, LLC""",null,"""706 OXFORD RD""",null,null,"""YPSILANTI""","""MI""",48197,2146,"""USA""",null,null,null,12,"""R""",73,7300,732990,73,7300,732990,59.008,"""MI**161""","""USA""",null,null,12,"""TO PROVIDE LOANS TO RESTORE AS…",null,null,null,null,28600.0,3895.32,"""NON"""
"""A""",20200803,7,2,2455458209,1,null,null,null,"""SAI Exempt""","""PINK PALETTE ARTISTS LLC""",null,"""7738 HERON LAKES DR""",null,null,"""HOUSTON""","""TX""",77064,1739,"""USA""",null,null,null,2,"""R""",73,7300,732990,73,7300,732990,59.008,"""TX**201""","""USA""",null,null,2,"""TO PROVIDE LOANS TO RESTORE AS…",null,null,null,null,98400.0,13402.08,"""NON"""


In [49]:

# manual_matches = [(2000003, 54187609)]
# manual_no_matches = [0,1,2,3,4, 2000000, 2000002, 2000004, 2000005, 2000006]

In [90]:
probable_no_matches = [0, 2000001,2000003]
no_matches = [1,2,3,4, 2000002, 2000004, 2000005, 2000006]

In [102]:
combined_df[3000][interesting_cols]

AWARDEEORRECIPIENTLEGALENTITYNAME,LEGALENTITYADDRLINE1,LEGALENTITYCITYNAME,LEGALENTITYSTATECD,LEGALENTITYZIP5
str,str,str,str,i64
"""Aro Coatings LLC""","""1425 Cornell Rd""","""Green Bay ""","""WI""",54313


In [97]:
combined_df.shape

(3765391, 45)

In [103]:
# Look for string 'ditzler' in company names
mask = company_names_lower_case.str.contains('aro coat')
potential_matches = apollo_df.filter(mask)
len(potential_matches)


0

In [100]:
mask.arg_true()

organization_name
u32
1248223
1816489
2069428
3694805


In [101]:

# print all rows (don't truncate)
# pl.Config.set_tbl_rows(None)
with pl.Config() as cfg:
    cfg.set_tbl_cols(-1)
    cfg.set_tbl_rows(-1)
    display(potential_matches[['organization_name', 'organization_hq_location_city', 'organization_hq_location_state', 'organization_hq_location_postal_code']])

organization_name,organization_hq_location_city,organization_hq_location_state,organization_hq_location_postal_code
str,str,str,str
"""Farnaz Global""","""Atlanta""","""Georgia""","""30306"""
"""Farnaz Advertising Asssociates""",null,null,null
"""FarnazFever""","""Mumbai""","""Maharashtra""","""400002"""
"""Farnaz Makeup & Hair Studio""",null,null,null


In [92]:
from rapidfuzz import process

In [95]:
import time

for sampled_name in sampled_names_lower_case:
    start = time.time()
    extracted = process.extractOne(sampled_name, company_names_lower_case, score_cutoff=90)
    print(f'Matched {sampled_name} to {extracted} in {time.time() - start} seconds')
# process.extractOne(sampled_names_lower_case[0], company_names_lower_case, score_cutoff=90)

Matched destiny construction team, inc. to ('construction construction', 95.0, 9280373) in 31.190216541290283 seconds
Matched hovhanes papazyan to ('yan', 90.0, 9233834) in 42.667500734329224 seconds
Matched invisible ink concierge services to ('invisible ink', 90.0, 378104) in 51.62486243247986 seconds
Matched custom glass of conway, inc. to ('custom glass', 90.0, 4834370) in 50.453418493270874 seconds
Matched rydate llc to ('ate', 90.0, 5145125) in 34.29480004310608 seconds
Matched davis unique services to ('unique services', 95.0, 1685182) in 35.82584547996521 seconds
Matched kailash investments llc to ('kilashee investments llc', 93.61702127659575, 40414668) in 40.041024684906006 seconds


KeyboardInterrupt: 

In [99]:
company_names_lower_case    

Company Name
str
"""comtech"""
"""tupperware"""
"""bion diagnostic sciences inc"""
"""garden cafe"""
"""garden cafe"""
…
"""county of hillsborough"""
"""crabtree & evelyn"""
"""alternative communication"""


In [107]:
4000000/3600

1111.111111111111

In [98]:
company_names_lower_case = pl.Series(company_names_lower_case)


In [105]:
import time

# If the company names are longer than 10 characters then drop the last 5 characters
# company_names_lower_case_shortened = company_names_lower_case.map(lambda x: x if len(x) <= 10 else x[:-5])
company_names_lower_case_shortened = company_names_lower_case.apply(lambda x: x if len(x) <= 10 else x[:-5])


for sampled_name in sampled_names_lower_case:
    start = time.time()
    # Find a perfect match in the company names if it exists

    # Filter to find matches
    matches = company_names_lower_case.filter(company_names_lower_case == sampled_name)

    # Check if matches are not empty and get the first result
    if matches.len() > 0:
        first_match = matches[0]
        print(f'First match: {first_match} for {sampled_name} in {time.time() - start} seconds')
    else:
        # If string is longer than 10 characters retry with a substring
        sampled_name_shortened = sampled_name if len(sampled_name) <= 10 else sampled_name[:-5]
        matches = company_names_lower_case_shortened.filter(company_names_lower_case_shortened == sampled_name_shortened)
        print(f'No matches found for {sampled_name} in {time.time() - start} seconds, retrying')
        
        if matches.len() > 0:
            first_match = matches[0]
            print(f'Shortened match: {first_match} for {sampled_name} in {time.time() - start} seconds')
            
        
    # extracted = process.extractOne(sampled_name, company_names_lower_case, score_cutoff=90)
    # print(f'Matched {sampled_name} to {extracted} in {time.time() - start} seconds')
# process.extractOne(sampled_names_lower_case[0], company_names_lower_case, score_cutoff=90)

AttributeError: 'Series' object has no attribute 'apply'

In [114]:
4*4*3 * 7.48

359.04

In [ ]:
import polars as pl
from fuzzywuzzy import process
import random

# Set seed for reproducibility in sampling
random.seed(42)

# Sample 100 random names from 'combined_df' and convert to lowercase
sampled_name = combined_df['AWARDEEORRECIPIENTLEGALENTITYNAME'].sample(n=100, with_replacement=False)#.apply(lambda x: x.lower())

print("A")

# Convert Polars DataFrame to pandas for fuzzy matching
sampled_names_pandas = sampled_name.to_pandas().str.lower()
company_names_pandas = b2b_df['Company Name'].to_pandas().str.lower()
print("B")
# Define a function to find the best match in 'b2b_df'
def get_best_match(name):
    best_match, score = process.extractOne(name, company_names_pandas)
    print(f"Legal Name: {legal_name} -> Best Match Company Name: {company_name}")

    return best_match

# Apply fuzzy matching
matches = sampled_names_pandas.apply(get_best_match)

# Print results
for legal_name, company_name in zip(sampled_names_pandas, matches):
    print(f"Legal Name: {legal_name} -> Best Match Company Name: {company_name}")


In [79]:
import polars as pl
from fuzzywuzzy import process
import random

# Set seed for reproducibility in sampling
random.seed(42)

# Sample 100 random names from 'combined_df' and convert to lowercase
sampled_name = combined_df['AWARDEEORRECIPIENTLEGALENTITYNAME'].sample(n=100, with_replacement=False)#.apply(lambda x: x.lower())

print("A")

# Convert Polars DataFrame to pandas for fuzzy matching
sampled_names_pandas = sampled_name.to_pandas().str.lower()
company_names_pandas = b2b_df['Company Name'].to_pandas().str.lower()
print("B")
# Define a function to find the best match in 'b2b_df'
def get_best_match(name):
    best_match, score = process.extractOne(name, company_names_pandas)
    print(f"Legal Name: {legal_name} -> Best Match Company Name: {company_name}")

    return best_match

# Apply fuzzy matching
matches = sampled_names_pandas.apply(get_best_match)

# Print results
for legal_name, company_name in zip(sampled_names_pandas, matches):
    print(f"Legal Name: {legal_name} -> Best Match Company Name: {company_name}")


A
B


KeyboardInterrupt: 

In [48]:

b2b_df = pd.concat([pd.read_csv(file) for file in b2b_csv_files])

/tmp/ipykernel_60045/3965579054.py:1: DtypeWarning: Columns (6,15) have mixed types. Specify dtype option on import or set low_memory=False.
  b2b_df = pd.concat([pd.read_csv(file) for file in b2b_csv_files])
/tmp/ipykernel_60045/3965579054.py:1: DtypeWarning: Columns (6,15) have mixed types. Specify dtype option on import or set low_memory=False.
  b2b_df = pd.concat([pd.read_csv(file) for file in b2b_csv_files])
/tmp/ipykernel_60045/3965579054.py:1: DtypeWarning: Columns (6,15) have mixed types. Specify dtype option on import or set low_memory=False.
  b2b_df = pd.concat([pd.read_csv(file) for file in b2b_csv_files])
/tmp/ipykernel_60045/3965579054.py:1: DtypeWarning: Columns (6,15) have mixed types. Specify dtype option on import or set low_memory=False.
  b2b_df = pd.concat([pd.read_csv(file) for file in b2b_csv_files])
/tmp/ipykernel_60045/3965579054.py:1: DtypeWarning: Columns (6,15) have mixed types. Specify dtype option on import or set low_memory=False.
  b2b_df = pd.concat([p

In [49]:
b2b_df

,Company Name,Address,City,State,Zip,County,Phone,Contact First,Contact Last,Title,Direct Phone,Email,Website,Employee Count,Annual Sales,SIC Code,Industry
0,Comtech,9215 151st Ave Ne,Redmond,WA,98052-3511,King,4254518138,Eleizer,Scharf,President,NaN,escharf@comtech-group.com,comtechphones.com,10 To 19,$20 To 50 Million,506563,Telecommunication Equip/Syst-Whol/Mfrs
1,Tupperware,916 S 30th Ave,Yakima,WA,98902-4074,Yakima,5099659224,Fred,Lubarsky,Owner,NaN,fsl31@aol.com,aol.com,1 To 4,"Less Than $500,000",596304,Home Demonstration-Merchandise
2,Bion Diagnostic Sciences Inc,12277 134th Ct Ne Ste 100,Redmond,WA,98052-2431,King,4258211010,Kelly,Walsh,Manager,NaN,kwalsh@polymedco.com,polymedco.com,NaN,NaN,289900,"Chemical Preparations, Nec"
3,Garden Cafe,18923 Peter Johnson Rd,Mount Vernon,WA,98273-9302,Skagit,3608489189,Martin,Tynan,Manager,NaN,mtynan@gardencafe.com,gardencafe.com,5 To 9,"Less Than $500,000",581208,Restaurants
4,Garden Cafe,18923 Peter Johnson Rd,Mount Vernon,WA,98273-9302,Skagit,3608489189,Dao,Choi,Owner,NaN,dchoi@gardencafe.com,gardencafe.com,5 To 9,"Less Than $500,000",581208,Restaurants
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
274082,County Of Hillsborough,400 Mast Rd,Goffstown,NH,03045-2427,Hillsborough,6036275540.0,Richard,Kerby,Manager,8132725625.0,richard@hillsboroughcounty.org,hillsboroughcountynh.org,1 To 4,Unknown,912103,Government Offices-County
274083,Crabtree & Evelyn,2 Common Ct Unit B8,North Conway,NH,03860-5440,Carroll,6033564100.0,Romona,Ropper,Manager,NaN,rropper@crabtree-evelyn.com,crabtree-evelyn.com,5 To 9,$1 To 2.5 Million,599992,Cosmetics & Perfumes-Retail
274084,Alternative Communication,3 Bud Way Ste 20,Nashua,NH,03063-1700,Hillsborough,6038823100.0,Warren,Kane,Manager,NaN,warren@urnet.com,acstelcom.com,1 To 4,$1 To 2.5 Million,489903,Communications
274085,Alternative Communication,3 Bud Way Ste 20,Nashua,NH,03063-1700,Hillsborough,6038823100.0,Terri,Richards,Owner,NaN,terri@urnet.com,acstelcom.com,1 To 4,$1 To 2.5 Million,489903,Communications
